In [1]:
"""初始化"""

import pandas as pd
import numpy as np

stocks_hkd = pd.read_csv('csv/stocks-hkd.csv', dtype={'Symbol':str})
trades_hkd = pd.read_csv('csv/trades-hkd.csv', dtype={'Symbol':str})
dividends_hkd = pd.read_csv('csv/dividends-hkd.csv', dtype={'Symbol':str})
stocks_hkd.set_index('Symbol', inplace=True)
trades_hkd.set_index('Symbol', inplace=True)
dividends_hkd.set_index('Symbol', inplace=True)

In [2]:
"""
为每笔交易计算实际金额

买入交易：买入价格 * 买入股数 + 佣金 + 税
卖出交易：卖出价格 * 卖出股数 - 佣金 - 税
"""

proceed = trades_hkd['TradePrice'] * trades_hkd['Qty'].abs()
fee = trades_hkd['Commission'] + trades_hkd['Tax']
trades_hkd['Basis'] = np.where(trades_hkd['Direction']=='BUY', proceed+fee, proceed-fee)

In [3]:
"""为所有交易过的股票建立DataFrame，并为每只股票计算持仓股数"""

#对所有的交易记录按股票分组
group_trade = trades_hkd.groupby(trades_hkd.index)

stock_qty = group_trade['Qty'].sum()
stocks_hkd_summary = stocks_hkd.join(stock_qty)

In [4]:
"""为每只股票计算买入股数，卖出股数"""

group_trade = trades_hkd.groupby([trades_hkd.index, trades_hkd['Direction']])
buyorsell_qty = group_trade['Qty'].sum().unstack(fill_value=0)
stocks_hkd_summary = stocks_hkd_summary.join(buyorsell_qty)
stocks_hkd_summary.columns = ['Name', 'Currency', 'Qty', 'Buy_Qty', 'Sell_Qty']

In [5]:
"""为每只股票计算总买入额，总卖出额，平均买入价格，平均卖出价格和已实现盈亏"""

buyorsell_basis = group_trade['Basis'].sum().unstack()
stocks_hkd_summary['Buy_Basis'] = buyorsell_basis['BUY']
stocks_hkd_summary['Sell_Basis'] = buyorsell_basis['SELL']
stocks_hkd_summary['Buy_Cost'] = stocks_hkd_summary['Buy_Basis']/stocks_hkd_summary['Buy_Qty']
stocks_hkd_summary['Sell_Cost'] = stocks_hkd_summary['Sell_Basis']/stocks_hkd_summary['Sell_Qty'].abs()
stocks_hkd_summary['Realized_PnL'] = stocks_hkd_summary['Sell_Basis'] - stocks_hkd_summary['Buy_Cost']*stocks_hkd_summary['Sell_Qty'].abs()

In [6]:
"""为每只股票计算累计分红"""

dividends_hkd['Dividend'] = dividends_hkd['PerShare']*dividends_hkd['Qty']-dividends_hkd['Commission']-dividends_hkd['Tax']

#对所有的分红记录按股票分组
group_dividend = dividends_hkd.groupby(dividends_hkd.index)

#stock_dividend = group_dividend['Dividend'].sum()
stocks_hkd_summary['Dividend'] = group_dividend['Dividend'].sum()


In [7]:
"""对于已实现盈亏和分红用零值替换NaN"""

stocks_hkd_summary['Realized_PnL'].fillna(0.0, inplace=True)
stocks_hkd_summary['Dividend'].fillna(0.0, inplace=True)

In [8]:
"""定义获取股票最新报价的函数"""

import datetime
import pandas_datareader.data as web
import tushare as ts

def getYahooQuote(symbols):
    start = datetime.datetime.today()
    end = start
    
    quotes_dict = {}
    yahoo_symbols = symbols + '.HK'
    pnl = web.DataReader(yahoo_symbols, 'yahoo', start, end)
    df = pnl['Close'].T
    df.index.name = 'Symbol'
    df.columns=['Last']
    df.index = df.index.map(lambda x: x.rstrip('.HK'))
    
    return df


def getTuShareQuote(symbols):
    cons = ts.get_apis()
    last = datetime.datetime.today() - datetime.timedelta(days=7)
    
    quotes_dict = {}
    tushare_symbols = '0' + symbols
    for symbl in tushare_symbols:
        df = ts.bar(symbl, conn=cons, asset='X', start_date=last, end_date='')
        quotes_dict[symbl[1:]] = df['close'].iloc[0]

    quotes = pd.Series(quotes_dict)
    return quotes

In [9]:
"""为当前持仓股票获取最近价格"""

stocks_hkd_hold = stocks_hkd_summary.loc[lambda df: df.Qty > 0]
stock_last = getYahooQuote(stocks_hkd_hold.index)
stocks_hkd_summary['Last'] = stock_last

In [10]:
"""为未平仓股票计算未实现盈亏"""

stocks_hkd_summary['Unrealized_PnL'] = stocks_hkd_summary['Qty'] * (stocks_hkd_summary['Last'] - stocks_hkd_summary['Buy_Cost'])
#对未已实现盈亏和分红用零值替换NaN
stocks_hkd_summary['Unrealized_PnL'].fillna(0.0, inplace=True)
stocks_hkd_summary['Earning'] = stocks_hkd_summary['Dividend'] + stocks_hkd_summary['Realized_PnL'] + stocks_hkd_summary['Unrealized_PnL']

In [11]:
'''为每只股票生成最终盈亏总结'''

stocks_final = stocks_hkd_summary[['Name', 'Last', 'Qty', 'Dividend', 'Realized_PnL', 'Earning']]
stocks_final.loc[lambda df: df.Qty > 0].round(2)

,Name,Last,Qty,Dividend,Realized_PnL,Earning
Symbol,,,,,,
1988,民生银行,7.41,12000,644.96,2834.05,-6213.58
0512,远大医药,2.82,32000,0.00,-9048.17,23275.54
1918,融创中国,34.85,10000,15459.23,792833.80,1085248.16
2601,中国太保,37.75,6000,0.00,2533.30,20465.38
2666,环球医疗,7.89,20000,0.00,0.00,23594.47
0392,北京控股,46.10,4000,1838.50,0.00,20927.48
0656,复星国际,18.78,10000,0.00,-460.09,930.70
0735,中国电力新能源,4.68,35000,0.00,0.00,-340.88
3883,中国奥园,4.38,20000,0.00,0.00,299.79
